In [7]:
#simulate brownian motion
from scipy.stats import norm
import math

def sim_brownian_motion(T,N):
    dt=T/N
    sqrdt=math.sqrt(dt)
    x=0
    for k in range(N):
        print(round(k*dt,2),x)
        x=x+sqrdt*norm.rvs()

In [8]:
sim_brownian_motion(1,20)

0.0 0
0.05 0.05073573644246874
0.1 0.002090768998819513
0.15 -0.020860552154988984
0.2 -0.3496373644217584
0.25 0.03340153886020997
0.3 -0.02512624153773025
0.35 0.15652442599536814
0.4 0.31647080101009917
0.45 0.05479551933458798
0.5 -0.15052643505807317
0.55 -0.14483989367807645
0.6 -0.135042160307088
0.65 0.09285807189063386
0.7 0.12850406119729452
0.75 0.05414454528902665
0.8 -0.1044162831841948
0.85 -0.1983217003165374
0.9 -0.13053507252883467
0.95 -0.16514444954880775


In [14]:
#simulate GBM
def sim_gbm(T,N,S,mu,sigma):
    '''
    mu: mean of return
    sigma: std of return
    '''
    dt=T/N
    sigsqrdt=sigma*math.sqrt(dt)
    drift=(mu-0.5*sigma**2)*dt
    logs=math.log(S)
    for k in range(N):
        print(round(k*dt,2),math.exp(logs))
        logs=logs+drift+sigsqrdt*norm.rvs()

In [15]:
sim_gbm(1,20,50,1,0.05)

0.0 49.99999999999999
0.05 51.27567014557494
0.1 54.50781220083841
0.15 57.73992407739632
0.2 60.085910791922394
0.25 63.96447675875789
0.3 66.89793328426114
0.35 69.92277310954645
0.4 73.31374022253354
0.45 76.7038704769544
0.5 79.91783171212616
0.55 84.42236305209124
0.6 90.24208515961027
0.65 93.17613566963459
0.7 97.49801323429854
0.75 103.7835483492889
0.8 108.40767884023123
0.85 115.05952742308398
0.9 121.7616117899563
0.95 130.03484794348185


Could use Numpy cumsum to accelerate the process
[Brownian Motion](https://scipy-cookbook.readthedocs.io/items/BrownianMotion.html)
[Geometric Brownian Motion](https://towardsdatascience.com/simulating-stock-prices-in-python-using-geometric-brownian-motion-8dfd6e8c6b18)

In [17]:
#black-sholes call
import numpy as np
from scipy.stats import norm

def BS_Call(S,K,r,sigma,q,T):
    d1=(np.log(S/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2=d1-sigma*np.sqrt(T)
    call=S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
    return call

In [18]:
BS_Call(50,40,0.05,0.3,0.02,2)

14.459572400816924

In [21]:
#Europe Call_GARCH_Monte Carlo
import numpy as np
from scipy.stats import norm

def Eur_Call_GARCH_MC(S, K, r, sigma0, q, T, N, kappa, theta, lamda, M):
    '''
             S = initial stock price
'            K = strike price
'            r = risk-free rate
'            sigma0 = initial volatility
'            q = dividend yield
'            T = time to maturity
'            N = number of time periods
'            kappa = GARCH parameter
'            theta = GARCH parameter
'            lambda = GARCH parameter
'            M = number of simulations
    '''
    
    dt=T/N
    sqrdt=np.sqrt(dt)
    a=kappa*theta
    b=(1-kappa)*lamda
    c=(1-kappa)*(1-lamda)
    payoffs=[]
    for i in range(M):
        logS=np.log(S)
        sigma=sigma0
        for j in range(N):
            y=sigma*norm.rvs()
            logS+=(r-q-0.5*sigma**2)*dt+sqrdt*y
            sigma=np.sqrt(a+b*y**2+c*sigma**2)
        payoffs.append(np.maximum(np.exp(logS)-K,0))
    price = np.exp(-r*T)*sum(payoffs)/float(M)
    return round(price,4)

In [22]:
Eur_Call_GARCH_MC(50,40,0.05,0.3,0.02,2,24,0.5,0.09,0.5,1000)

14.1607

In [5]:
#Europe Call_Binomial
import numpy as np

def Eur_Call_Binomial(S0,K,r,sigma,q,T,N):
    dt=T/N
    u=np.exp(sigma*np.sqrt(dt))
    d=1/u
    pu=(np.exp((r-q)*dt)-d)/(u-d)
    pd=1-pu
    
    #start from the bottom node at last date
    S=S0*d**N
    prob=pd**N
    CallV=np.maximum(S-K,0)*prob
    
    #step up over nodes at last date
    for i in range(1,N):
        S=S*u**2
        prob=prob*(pu/pd)*(N-i+1)/i
        CallV=CallV+prob*np.maximum(S-K,0) #sum weighted values at last date
    
    price = np.exp(-r*T)*CallV
    return round(price,4)

In [6]:
import timeit
start_time=timeit.default_timer()
Eur_Call_Binomial(50,40,0.05,0.3,0.02,2,50)
print(timeit.default_timer()-start_time,"seconds")

0.00055882100002691 seconds


Euro_Binomial Option Pricing with GARCH and classes: [Binomial Option Pricing Model](https://medium.com/engineer-quant/binomial-option-pricing-model-5e6b9e91c7da)

In [15]:
#American Put_Binomial
def Ame_Put_Binomial(S0,K,r,sigma,q,T,N):
    dt=T/N
    u=np.exp(sigma*np.sqrt(dt))
    d=1/u
    pu=(np.exp((r-q)*dt)-d)/(u-d)
    pd=1-pu
    
    #start from the bottom node at last date
    #S=S0*d**N
    #PutV=np.maximum(K-S,0)
    #step up over nodes at last date
    
    #put values at the last date
    PutV=np.asarray([np.maximum(K-S0*u**j*d**(N-j),0) for j in range(N+1)])
    
    #backward
    stock = np.asarray([K-S0*u**j*d**(N-j) for j in range(N+1)])
    for i in range(N-1,-1,-1):
        stock[:]=stock[:]*u
        PutV[:-1]=np.maximum(stock[:],np.exp(-r*dt)*(pu*PutV[1:]+pd*PutV[:-1])) #some problems here with the row numbers
        
    return PutV[0]

In [16]:
Ame_Put_Binomial(50,40,0.05,0.3,0.02,2,50)

ValueError: operands could not be broadcast together with shapes (51,) (50,) 

[American Option Pricing](http://gosmej1977.blogspot.com/2013/02/american-options.html)